In [1]:
from sklearn.metrics import confusion_matrix,precision_recall_fscore_support
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer , WordNetLemmatizer
from sklearn.utils import shuffle
from nltk.corpus import stopwords
import pandas as pd 
import numpy as np
import string

In [2]:
data = pd.read_csv(r"spam.csv", encoding='latin-1')
data.head()
data = shuffle(data)
inp = data.iloc[:,1:2]
outp = data.iloc[:,0:1]
split = .8
train_data, train_label = inp.iloc[:(int)(inp.shape[0]*split),:] , outp.iloc[:(int)(inp.shape[0]*split),:] 
test_data, test_label = inp.iloc[(int)(inp.shape[0]*split):,:] , outp.iloc[(int)(inp.shape[0]*split):,:]

# train_data, train_label = inp, outp
# test_data, test_label = inp, outp

outp.head()

,v1
4438,ham
125,ham
547,ham
2591,ham
2666,ham


In [3]:
# lemma = WordNetLemmatizer()
ps = PorterStemmer()
def preprocess_data(mess):
#     return mess
    mess = mess.lower()
    nostop = " ".join([i for i in mess.split() if i not in stopwords.words('english')])
    nonum = [char for char in nostop if char not in ['0','1','2','3','4','5','6','7','8','9']]
    nopunc =[char for char in nonum if char not in string.punctuation]
    nopunc=''.join(nopunc)
    stem = " ".join(ps.stem(word) for word in nopunc.split())
#     lem = " ".join(lemma.lemmatize(word) for word in nopunc.split())
    return stem

In [4]:
train_data['v2'] = train_data['v2'].apply(preprocess_data)
print(train_data.shape)
train_data.head()

(4457, 1)


C:\Users\Narahc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,v2
4438,want grasp pretti booti
125,good stuff do
547,sorri took long omw
2591,still work go onit small hous
2666,nice day and impress sensibl went home earli f...


In [5]:
print(inp.head())
nham = (train_label.v1 == 'ham').sum()
nspam = (train_label.v1 == 'spam').sum()
ntot = train_label.shape[0]
print("Ham :", nham)
print("Spam :", nspam)
print("Total :", ntot)

                                                     v2
4438                            want grasp pretti booti
125                                       good stuff do
547                                 sorri took long omw
2591                      still work go onit small hous
2666  nice day and impress sensibl went home earli f...
Ham : 3860
Spam : 597
Total : 4457


In [6]:
vectorizer = CountVectorizer()
vectorizer.fit(train_data['v2'])

spam_ar = train_data[(train_label.v1 == 'spam')] 
ham_ar = train_data[(train_label.v1 == 'ham')]

spam_vec = vectorizer.transform(spam_ar['v2'])
ham_vec = vectorizer.transform(ham_ar['v2'])
spam_freq = np.array(spam_vec.sum(axis=0))
ham_freq = np.array(ham_vec.sum(axis=0))

no_spam = len(spam_ar)
no_ham = len(ham_ar)

spam_freq = spam_freq.reshape(spam_freq.shape[1])
ham_freq = ham_freq.reshape(ham_freq.shape[1])
tot_freq = spam_freq + ham_freq

spam_words = spam_vec.sum() 
ham_words = ham_vec.sum()
tot_words = spam_words + ham_words

print(len(spam_ar))
print(len(ham_ar))
print(tot_words)

597
3860
39229


In [7]:
def predict(predis):
    predis = preprocess_data(predis)
    spam_num = 1.00
    ham_num = 1.00
    for word in predis.split():
        
        fspam = 0
        fham = 0
        if word in vectorizer.vocabulary_:
            fspam = spam_freq[vectorizer.vocabulary_[word]]
            fham = ham_freq[vectorizer.vocabulary_[word]]

        spam_num *= ((fspam +1) / (spam_words + len(vectorizer.vocabulary_)))
        ham_num *= ((fham +1) / (ham_words + len(vectorizer.vocabulary_)))

    spam_num = (spam_num * ((no_spam)/ ( no_spam + no_ham)))
    ham_num = (ham_num * ((no_ham) / ( no_spam + no_ham)))
    spamm = spam_num / (spam_num + ham_num)
    hamm = ham_num / (spam_num + ham_num)
    if spamm >= hamm:
        return "spam"
    return "ham"

In [8]:
actual =[]
prediction =[]
# test_data['v2'] = test_data['v2'].apply(preprocess_data)
test_data.head()
for i, row in test_data.iterrows():
    actual.append(predict(test_data['v2'][i]))
    prediction.append(test_label['v1'][i])
M = confusion_matrix(actual,prediction)
print("Confusion Matrix :", M)
print("ACCURACY :",(M[0][0]+M[1][1])/((M[0][0]+M[0][1] + M[0][1]+M[1][1])))
print(precision_recall_fscore_support(actual,prediction))

Confusion Matrix : [[949  12]
 [ 16 138]]
ACCURACY : 0.9783978397839784
(array([0.98341969, 0.92      ]), array([0.98751301, 0.8961039 ]), array([0.9854621 , 0.90789474]), array([961, 154], dtype=int64))


In [9]:
predict(",I am hot n horny and willing I live local to you - text a reply to hear strt back from me 150p per msg Netcollex LtdHelpDesk: 02085076972 reply Stop to end")

'spam'